# GRASS GIS GUI intro

In this part we will demonstrate starting GRASS GIS, creating new project and basic data visualization.

## Start GRASS GIS

First, we create new empty location (project) called *dix_park* that uses projection [UTM zone 17 N](https://epsg.io/6346) with EPSG:6346.

We will use Location Wizard to create it. Afterwards we are automatically switched to the new location.

![Create new location](img/new_location.gif)

## Import data

_If you run this workshop online, data is already prepared in the environment. Otherwise download the data [here](http://fatra.cnr.ncsu.edu/foss4g2021/)._

We will import prepared digitial surface model (DSM), bare ground (digital terrain model, DTM) and ortho maps. 
The data CRS matches the CRS of the *dix_park* location, so we don't need to reproject it.

![Import raster data](img/import_raster.gif)


Next, we will import pre-downloaded OSM data of roads restricted to our study area. We obtained the roads using Overpass Turbo with this [query](https://overpass-turbo.eu/s/1aGu) and exported to GeoJSON. 

The data comes in EPSG:4326, so it will be automatically reprojected to UTM during the import.

![Import vector data](img/import_vector.gif)

## Data visualization

Now we first display the imported data bare ground elevation with the roads dataset, and then a road named _Umstead Drive_ in yellow on top of ortho:

![Display raster and vector data, set color](img/display.gif)

We can also visualize data in 3D. Here we drape the ortho over the DSM.

![3D visualization](img/3D_visualization.gif)

## GRASS GIS tools

GRASS functionality is available through tools (also called modules). There are over 500 different tools in the core distribution and over 300 addon tools that can be used to prepare and analyze data.

Tools respect the following naming conventions:

Prefix | Function | Example
------ | -------- | -------
r.* | raster processing | r.mapcalc: map algebra
v.*	| vector processing	| v.clean: topological cleaning
i.*	| imagery processing | i.segment: object recognition
db.* | database management | db.select: select values from table
r3.* | 3D raster processing | r3.stats: 3D raster statistics
t.* | temporal data processing | t.rast.aggregate: temporal aggregation
g.* | general data management | g.rename: renames map
d.* | display | d.rast: display raster map

Note also that some tools have multiple dots in their names. For example, tools staring with v.net.* deal with vector network analysis and r.in.* tools import raster data into GRASS GIS spatial database.

### Finding and running a module

![Find modules](img/search_module.gif)

To find a tool for your analysis, type the term into the **search box within the Tools tab** in the Layer Manager or just browse the tool tree.
Most tools are also available from the **main menu**. For example, to find information about a raster map, use: *Raster → Reports and statistics → Basic raster metadata*.

If you already know the name of the tool, you can just use it in the command line. The GUI offers a **Console tab with command line** specifically built for running GRASS GIS tools. If you type the tool name there, you will get suggestions for automatic completion of the name. After pressing Enter, you will get the GUI dialog for the tool.


### GUI  vs command line vs Python

GRASS tools can be executed either through the GUI, command line or Python interfaces. The GUI offers a user-friendly approach to execute tools where the user can navigate to data layers that they would like to analyze and modify processing options with simple check boxes.
The GUI also offers an easily accessible manual on how to execute a tool. The command line interface allows users to execute a tool using command prompts specific to that tool. This is handy when you are running similar analyses with minor modification or are familiar with the tool options for quick efficient processing.

In this example we will show how the same tool can be executed with the GUI, in shell and in Python:

![v.extract](img/v.extract.png) &nbsp;&nbsp;&nbsp;&nbsp;
![v.extract](img/v.extract.gif)

The shell representation looks like this:
```
v.extract input=roads where="name = 'Umstead Drive'" output=umstead_drive_segments
```
And in Python you would call:
```
gs.run_command("v.extract", input="roads", where="name = 'Umstead Drive'", output="umstead_drive_segments")
```

## Computational region

Computational region is an important raster concept in GRASS GIS.
Before we use a raster tool to compute a new raster map, we must properly set the computational region. All raster computations will be performed in the specified extent and with the given resolution.
Among other things, this allows us to easily subset larger extent data for quicker testing of analysis, or to run an analysis of specific regions given by e.g. administrative units.

A few points to keep in mind:

 * computational region is defined by region extent and raster resolution
 * applies to all raster operations and vector to raster operations
 * persists between GRASS sessions, can be different for different mapsets
 * advantages: keeps your results consistent, avoids clipping, for computationally demanding tasks set region to smaller extent, check that your result is good and then set the computational region to the entire study area and rerun analysis
 * run `g.region -p` or in menu *Settings - Region - Display region* to see current region settings
 
 ![Computational region](img/region.gif)

The most common way to set region is **based on a raster map** - both extent and resolution. If the raster is added to Layer Manager, right click on it and select *Set computational region from selected map*.

```
g.region -p raster=dsm
```

Computational region can be set also **using a vector map**. In that case, only extent is set (as vector maps do not have any resolution - at least not in the way raster maps do). In the GUI, this can be done in the same way as for the raster map. In the command line, it looks like this:
```
g.region -p vector=roads
```

However now the resolution was adjusted based on the extent of the vector map, it is no longer a nice rounded number. If that's not desired, we can set it explicitly using -a flag and parameter res. Now the resolution is aligned to even multiples of 2 (the units are the units of the current location, in our case meters):
```
g.region -p vector=roads res=2 -a
```

Often we need to set the computational extent based on a vector map, but take the resolution and alignment from a raster map:
```
g.region -p vector=roads align=dsm
```

## Python API

There are two Python APIs for accessing a module's functionality - [GRASS GIS Python Scripting Library](https://grass.osgeo.org/grass-stable/manuals/libpython/script_intro.html) and [PyGRASS](https://grass.osgeo.org/grass78/manuals/libpython/pygrass_index.html).
PyGRASS is advantageous for more advanced workflows. Here we will be using Python Scripting Library (`import grass.script as gs`)
as it is simple and straightforward to use.
 

GRASS GIS Python Scripting Library provides functions to call GRASS modules within scripts as subprocesses. The most often used functions include:

 * [run_command()](https://grass.osgeo.org/grass-stable/manuals/libpython/script.html#script.core.run_command): used with modules which output raster/vector data where text output is not expected
 * [read_command()](https://grass.osgeo.org/grass-stable/manuals/libpython/script.html#script.core.read_command): used when we are interested in text output
 * [parse_command()](https://grass.osgeo.org/grass-stable/manuals/libpython/script.html#script.core.parse_command): used with modules producing text output as key=value pair
 * [write_command()](https://grass.osgeo.org/grass-stable/manuals/libpython/script.html#script.core.write_command): for modules expecting text input from either standard input or file


It also provides several wrapper functions for often called modules. The list of convenient wrapper functions with examples includes:

 * Raster metadata using [raster_info()](https://grass.osgeo.org/grass-stable/manuals/libpython/script.html#script.raster.raster_info): `gs.raster_info('dsm')`
 * Vector metadata using [vector_info()](https://grass.osgeo.org/grass-stable/manuals/libpython/script.html#script.vector.vector_info): `gs.vector_info('roads')`
 * List raster data in current location using [list_grouped()](https://grass.osgeo.org/grass-stable/manuals/libpython/script.html#script.core.list_grouped): `gs.list_grouped(type=['raster'])`
 * Get current computational region using [region()](https://grass.osgeo.org/grass-stable/manuals/libpython/script.html#script.core.region): `gs.region()`
 * Run raster algebra using [mapcalc()](https://grass.osgeo.org/grass-stable/manuals/libpython/script.html#script.raster.mapcalc): `gs.mapcalc()`

**Note for GUI:** The simplest way to execute the Python code which uses GRASS GIS packages is to use the Simple Python editor integrated in GRASS GIS (accessible from the toolbar or the Python tab in the Layer Manager). Another option is to write the Python code in your favorite plain text editor like Notepad++ (note that Python editors are plain text editors). Then run the script in GRASS GIS using the main menu *File -> Launch script*.

![Run Python in GUI](img/python.gif)